<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Tagtog_Preprocessing_221206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
!pip install oauth2client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

# from transformers import BertTokenizer

import json
import glob
import re
import os
import ast
import html
import requests

import zipfile
import shutil

import pandas as pd

from google.colab import files

# Tagtog Log-In

In [3]:
#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'Railway_Annotation.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [4]:
#압축파일 풀기
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("Railway_Annotation.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [5]:
#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#context list   # context == plain.html
context_name_list = os.listdir(folder_name + "plain.html/pool")
print(context_name_list)

#relation 폴더 경로   # relation == ann.json
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]

#anntation_lenged 정보
annotations_legend = folder_name + "annotations-legend.json"
with open(annotations_legend,"r") as f:
    annotations_legend = json.load(f)


['awUyBdL4lPpVlBOPjSzMhAuRk81a-20181208.pdf.plain.html', 'aPSRwvP5rtizz2O093I3oFX9uzjS-20180818.pdf.plain.html', 'aVuARZFRfgJMD0E67Coa.7G8Z9Pe-20200226.pdf.plain.html', 'arqduk0rnomRe20HKEeXlVPK8qx8-20191022.pdf.plain.html', 'a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html']


# Tagtog Annotation json File Preprocessing

In [10]:
def split_relation_start(text):
  text = text.split('|')[-1]
  start = text.split(',')[0]
  start = int(start)
  return start

In [67]:
'''
Loc : Sub, Obj 위치
Cls : Sub, Obj 클래스
Part : Sub, Obj html 상 문단
Sub : Sub 내용
Obj : Obj 내용
Label : 관계 있으면 1 없으면 0
Relation DataFrame Content
'''
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id for file_id in file_ids]
    context_files = [contexts_folder for file_id in file_ids]
    Rel_df = pd.DataFrame()
    text_df = pd.DataFrame()
    # Tagtog Annotation json 파일 데이터셋 전처리
    for relation_file, context_file, file_num in zip(relation_files, context_files, file_nums):
      with open(relation_file,"r") as f:
        relation = json.load(f)
        idx_list = []
        classid_list = []
        part_list = []
        text_list = []
        start_list = []
        doc_list = []

        for idx, content in enumerate(relation['entities']):
          doc_list.append(str(file_num)[2:-13])
          idx_list.append(idx)
          classid_list.append(content['classId'])
          part_list.append(content['part'])
          text_list.append(content['offsets'][0]['text'])
          start_list.append(content['offsets'][0]['start'])
          text_df = text_df.append(pd.DataFrame(zip(doc_list, start_list, classid_list, part_list, text_list)))
        df = pd.DataFrame(zip(idx_list, start_list, classid_list, part_list, text_list))
        df.columns = ['Index', 'Start','Class', 'Part', 'Text']

        doc_list = []
        loc_list = []
        cls_list = []
        part_list = []
        subject_list = []
        object_list = []
        label_list = []


        for rel in relation['relations']:
          subject = split_relation_start(rel['entities'][0])
          subject_class = df.loc[df['Start']==subject]['Class'].values[0]
          subject_part = df.loc[df['Start']==subject]['Part'].values[0]
          subject_text = df.loc[df['Start']==subject]['Text'].values[0]
          

          object = split_relation_start(rel['entities'][1])
          object_class = df.loc[df['Start']==object]['Class'].values[0]
          object_part = df.loc[df['Start']==object]['Part'].values[0]
          object_text = df.loc[df['Start']==object]['Text'].values[0]

          loc = str(subject) + '_' + str(object)
          cls = subject_class + '_' + object_class
          part = subject_part + '_' + object_part

          doc_list.append(str(file_num)[2:-13])
          loc_list.append(loc)
          cls_list.append(cls)
          part_list.append(part)
          subject_list.append(subject_text)
          object_list.append(object_text)
          label_list.append(1)
          Rel_df = Rel_df.append(pd.DataFrame(zip(doc_list, loc_list, cls_list, part_list, subject_list, object_list, label_list)))

Rel_df.columns = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']
text_df.columns = ['Doc', 'Start','Class', 'Part', 'Text']
print(Rel_df)
print(text_df)

       Doc          Loc       Cls       Part  \
0   200226      228_112   e_6_e_4  s1p5_s1p5   
0   200226      228_112   e_6_e_4  s1p5_s1p5   
1   200226      279_112   e_6_e_4  s1p5_s1p5   
0   200226      228_112   e_6_e_4  s1p5_s1p5   
1   200226      279_112   e_6_e_4  s1p5_s1p5   
..     ...          ...       ...        ...   
4     mple    3394_3607  e_22_e_4  s1p5_s1p5   
5     mple    3434_3607  e_22_e_4  s1p5_s1p5   
6     mple    3455_3607  e_22_e_4  s1p5_s1p5   
7     mple  13368_13094  e_23_e_6  s1p6_s1p6   
8     mple  13525_13057  e_23_e_6  s1p6_s1p6   

                                                  Sub  \
0      기지관제원이 선로전환기 할출 및 할입 사 고에 대한 업무 매뉴얼을 적용하지 않는 것   
0      기지관제원이 선로전환기 할출 및 할입 사 고에 대한 업무 매뉴얼을 적용하지 않는 것   
1   기지구내 선로의 선형이 입 환 작업시 출·입고 열차의 진로를 지장 하는 구조로 되어...   
0      기지관제원이 선로전환기 할출 및 할입 사 고에 대한 업무 매뉴얼을 적용하지 않는 것   
1   기지구내 선로의 선형이 입 환 작업시 출·입고 열차의 진로를 지장 하는 구조로 되어...   
..                                                ...   
4                 뒷 대차 첫

In [ ]:
print(Rel_df)

       Doc        Loc      Cls         Part  \
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
1   181208  4437_4249  e_6_e_4    s1p2_s1p2   
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
1   181208  4437_4249  e_6_e_4    s1p2_s1p2   
..     ...        ...      ...          ...   
2   200226     191_76  e_6_e_4  s1p17_s1p17   
0   200226    228_112  e_6_e_4    s1p5_s1p5   
1   200226    279_112  e_6_e_4    s1p5_s1p5   
2   200226     191_76  e_6_e_4  s1p17_s1p17   
3   200226     242_76  e_6_e_4  s1p17_s1p17   

                                                  Sub  \
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
1   청량신호소 21A호, 21B호 선로전환기를 설계변경하고 설치․시공ㆍ감리 과 정에 변...   
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
1   청량신호소 21A호, 21B호 선로전환기를 설계변경하고 설치․시공ㆍ감리 과 정에 변...   
..                                                ...   
2      기지관제원이 선로전환기 할출 및 할입 사 고에 대한 

In [9]:
context_name, relation_folder, contexts_folder = context_name_list[0], relation_folder_paths[0], contexts_folders_paths[0]
file_ids = os.listdir(relation_folder)[0]
file_nums = file_ids.split("-")[1]
relation_files = relation_folder + "/"+ file_ids
context_files = file_ids
context_df = pd.DataFrame()

with open(relation_files,"r") as f:
  relation = json.load(f)
  idx_list = []
  classid_list = []
  part_list = []
  text_list = []
  start_list = []

for idx, content in enumerate(relation['entities']):
  idx_list.append(idx)
  classid_list.append(content['classId'])
  part_list.append(content['part'])
  text_list.append(content['offsets'][0]['text'])
  start_list.append(content['offsets'][0]['start'])


{'annotatable': {'parts': ['s1p1', 's1p2', 's1p3', 's1p4', 's1p5', 's1p6', 's1p7', 's1p8', 's1p9', 's1p10', 's1p11', 's1p12', 's1p13', 's1p14', 's1p15', 's1p16', 's1p17', 's1p18', 's1p19', 's1p20', 's1p21', 's1p22']}, 'anncomplete': True, 'sources': [], 'metas': {}, 'entities': [{'classId': 'e_22', 'part': 's1p4', 'offsets': [{'start': 3874, 'text': '5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_4', 'part': 's1p5', 'offsets': [{'start': 112, 'text': '기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운전 지시를 받고, 선로전환기 상태를 확 인하지 않고 되돌이 운전을 시행한 것'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_6', 'part': 's1p5', 'offsets': [{'start': 228, 'text': '기지관제원이 선로전환기 할출 및 할입 사 고에 대한 업무 매뉴얼을 적용하지 않는 것'}], 'coordinates': [], 'confidence': {'state': 'pre-added

In [68]:
print(text_df)

       Doc  Start Class  Part  \
0   200226   3874  e_22  s1p4   
0   200226   3874  e_22  s1p4   
1   200226    112   e_4  s1p5   
0   200226   3874  e_22  s1p4   
1   200226    112   e_4  s1p5   
..     ...    ...   ...   ...   
9     mple  13368  e_23  s1p6   
10    mple  13525  e_23  s1p6   
11    mple  13608  e_23  s1p6   
12    mple  13689  e_23  s1p6   
13    mple  13763  e_23  s1p6   

                                                 Text  
0                 5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.  
0                 5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.  
1   기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운...  
0                 5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.  
1   기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운...  
..                                                ...  
9   화차 차축 베어링의 분해검수 주기와 교환 주기를 화차에 적재되는 화 물의 종류, 적...  
10  컨테이너를 화차에서 들어 올릴 때 화차가 함께 들어 올려지다 떨어지지 않도록 작업 ...  
11  화차의 발열 여부를 발견하기 위해 차축에 부착한 온도 테이프는 효과가 없는 것으로 ...  
12  역근무자가 열차 감시 업무를 적정하게 수행할 수 있도록 열차감시용 영상

#Google Spreadsheet 연동

In [14]:
# GOOGLE API KEY: json
key = files.upload()

Saving railway-tagtog-d24e6901ddb0.json to railway-tagtog-d24e6901ddb0.json


In [15]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './railway-tagtog-d24e6901ddb0.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = "annotation"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

In [ ]:
print(Rel_df.iloc[0, 4])

청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대로 표시되는 것을 확인하지 못한 것


In [ ]:
# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']
data = data[column_list]
data.head()

# 워크시트 범위 설정
worksheet.resize(len(Rel_df)+1,10)
list_range = f"a2:g{len(Rel_df)+1}"
cell_list = worksheet.range(list_range)

# 내용 작성
for i in range(len(cell_list)//len(column_list)):
    cell_list[(7*i)].value = str(Rel_df.iloc[i, 0])
    cell_list[(7*i)+1].value = str(Rel_df.iloc[i, 1])
    cell_list[(7*i)+2].value = str(Rel_df.iloc[i, 2])
    cell_list[(7*i)+3].value = str(Rel_df.iloc[i, 3])
    cell_list[(7*i)+4].value = str(Rel_df.iloc[i, 4])
    cell_list[(7*i)+5].value = str(Rel_df.iloc[i, 5])
    cell_list[(7*i)+6].value = int(Rel_df.iloc[i, 6])
    
worksheet.update_cells(cell_list)

{'spreadsheetId': '1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo',
 'updatedRange': 'annotation!A2:G192',
 'updatedRows': 191,
 'updatedColumns': 7,
 'updatedCells': 1337}

In [20]:
print(df)

    Index  Start Class  Part  \
0       0   3394  e_22  s1p5   
1       1   3434  e_22  s1p5   
2       2   3455  e_22  s1p5   
3       3   3607   e_4  s1p5   
4       4   3704   e_6  s1p5   
5       5   3741   e_6  s1p5   
6       6  12960   e_4  s1p6   
7       7  13057   e_6  s1p6   
8       8  13094   e_6  s1p6   
9       9  13368  e_23  s1p6   
10     10  13525  e_23  s1p6   
11     11  13608  e_23  s1p6   
12     12  13689  e_23  s1p6   
13     13  13763  e_23  s1p6   

                                                 Text  
0                 뒷 대차 첫 번째 차축과 두 번째 차축이 진행방향 좌측으로 탈선  
1                                     좌 측 세 번째 차축이 절손  
2                     탈선화차와 14번째 화차 사이의 공기호스 가 빠져있는 것  
3   차축에 설치되어 있는 차축 베어링 외륜의 내면과 롤러에 결함이 있는 상태에서 운행 ...  
4                 화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것  
5   차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...  
6   차축에 설치되어 있는 차축 베어링 외륜의 내면과 롤러에 결함이 있는 상태에서 운행 ...  
7                 화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것  
8   차축 베어링의 분해검

In [65]:
print(context_df)

       Doc  Start Class  Part  \
0   200226   3874  e_22  s1p4   
0   200226   3874  e_22  s1p4   
1   200226    112   e_4  s1p5   
0   200226   3874  e_22  s1p4   
1   200226    112   e_4  s1p5   
..     ...    ...   ...   ...   
9     mple  13368  e_23  s1p6   
10    mple  13525  e_23  s1p6   
11    mple  13608  e_23  s1p6   
12    mple  13689  e_23  s1p6   
13    mple  13763  e_23  s1p6   

                                                 Text  
0                 5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.  
0                 5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.  
1   기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운...  
0                 5번째 차량(3513호) 앞 대차가 선로 우측으 로 탈선하였다.  
1   기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운...  
..                                                ...  
9   화차 차축 베어링의 분해검수 주기와 교환 주기를 화차에 적재되는 화 물의 종류, 적...  
10  컨테이너를 화차에서 들어 올릴 때 화차가 함께 들어 올려지다 떨어지지 않도록 작업 ...  
11  화차의 발열 여부를 발견하기 위해 차축에 부착한 온도 테이프는 효과가 없는 것으로 ...  
12  역근무자가 열차 감시 업무를 적정하게 수행할 수 있도록 열차감시용 영상

In [70]:
#시트 선택하기
sheet_name = "context"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ['Index', 'Start', 'Class', 'Part', 'Text']
data.columns = column_list

# 워크시트 범위 설정
worksheet.resize(len(text_df)+1,10)
list_range = f"a2:e{len(text_df)+1}"
cell_list = worksheet.range(list_range)


print(len(cell_list)//len(column_list))
# 내용 작성
for i in range(len(cell_list)//len(column_list)):
    cell_list[(5*i)].value = str(text_df.iloc[i, 0])
    cell_list[(5*i)+1].value = str(text_df.iloc[i, 1])
    cell_list[(5*i)+2].value = str(text_df.iloc[i, 2])
    cell_list[(5*i)+3].value = str(text_df.iloc[i, 3])
    cell_list[(5*i)+4].value = str(text_df.iloc[i, 4])

worksheet.update_cells(cell_list)

1393


{'spreadsheetId': '1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo',
 'updatedRange': 'context!A2:E1394',
 'updatedRows': 1393,
 'updatedColumns': 5,
 'updatedCells': 6965}